<center> <h1>Proje - Derin öğrenme</h1></center>
<center> <h5> Fatma RIDAOUI </h5> </center>
<center><h6> 91220001225 </h6> </center>
    


In [71]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from PIL import ImageFile, Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import keras_tuner as kt
from keras_tuner import HyperParameters

In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


GPU found
2.9.0
Num GPUs Available:  1


In [102]:
list_train= os.listdir(r"C:\Users\zouhair\Desktop\Fatma's\OneDrive - Ege Üniversitesi\Ege_Dr_D2\Deep Learning\proje1\images\Train")

In [103]:
classes ={}
df_dataset= []
for each in list_train:
    className = each.split('(')[0].replace(" ","")
    df_dataset.append([each, className])
    if className not in classes:
        classes[className] = 0
    else:
        classes[className] = classes[className] +1

In [99]:
list_test = os.listdir(r"C:\Users\zouhair\Desktop\Fatma's\OneDrive - Ege Üniversitesi\Ege_Dr_D2\Deep Learning\proje1\images\Test")

In [128]:
test_classes ={}
df_dataset_test = []
i=0
for each in list_test:
    className = each.split('(')[0].replace(" ","")
    df_dataset_test.append([each, className])
    if i==5:
        break
    if className not in test_classes:
        i=i+1
        test_classes[className] = 0
        
    else:
        test_classes[className] = test_classes[className] +1
      


In [129]:
test_classes

{'100': 10, 'among': 10, 'angry': 10, 'ankh': 10, 'aroura': 0}

In [8]:
df = pd.DataFrame(df_dataset, columns=['file_name', 'class'])

In [9]:
test_df = pd.DataFrame(df_dataset_test, columns=['file_name', 'class'])

In [109]:
image_size = (224, 244)
batch_size= 16
input_shape=(224,244, 3)

In [110]:
"""model = ResNet50( include_top=False ,weights = 'imagenet', input_shape= input_shape , classes=num_classes)

for layer in model.layers[:-26]:
    layer.trainable = False
for layer in model.layers[-26:]:
    layer.trainable = True

x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
y = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

resnet_model = tf.keras.models.Model(inputs=model.input, outputs=y)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
resnet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history= resnet_model.fit(train_gen,
          validation_data=val_gen,               
          batch_size= batch_size,
          epochs=40)"""

"model = ResNet50( include_top=False ,weights = 'imagenet', input_shape= input_shape , classes=num_classes)\n\nfor layer in model.layers[:-26]:\n    layer.trainable = False\nfor layer in model.layers[-26:]:\n    layer.trainable = True\n\nx = tf.keras.layers.GlobalAveragePooling2D()(model.output)\nx = tf.keras.layers.Dense(512, activation='relu')(x)\nx = tf.keras.layers.Dropout(0.5)(x)\ny = tf.keras.layers.Dense(num_classes, activation='softmax')(x)\n\nresnet_model = tf.keras.models.Model(inputs=model.input, outputs=y)\noptimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)\nresnet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])\nhistory= resnet_model.fit(train_gen,\n          validation_data=val_gen,               \n          batch_size= batch_size,\n          epochs=40)"

In [111]:
def build_model(hp):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    
    for layer in base_model.layers[:-26]:
        layer.trainable = False
    for layer in base_model.layers[-26:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Define the hyperparameters to tune
    units = hp.Int('units', min_value=128, max_value=512, step=32)
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)
    learning_rate = hp.Float('learning_rate', min_value=1e-3, max_value=1e-1, sampling='log')

    # Add dense and dropout layers with the hyperparameters to tune
    x = Dense(units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    # Add the output layer with the appropriate number of units for your task
    num_classes = 95
    output = Dense(num_classes, activation='softmax')(x)

    optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)
    # Build and compile the model
    model = keras.models.Model(inputs=base_model.input, outputs=output)
    model.compile( optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [112]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='tuner_dir',
    project_name='resnet_model')

INFO:tensorflow:Reloading Tuner from tuner_dir\resnet_model\tuner0.json


In [113]:
path = "C:/Users/zouhair/Desktop/Fatma's/OneDrive - Ege Üniversitesi/Ege_Dr_D2/Deep Learning/proje1/images/Train"
test_path = "C:/Users/zouhair/Desktop/Fatma's/OneDrive - Ege Üniversitesi/Ege_Dr_D2/Deep Learning/proje1/images/Test"

datagen = ImageDataGenerator( rescale= 1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(dataframe=df, 
                                        directory= path,
                                        x_col= 'file_name',
                                        y_col= 'class',
                                        class_mode= "categorical",
                                        color_mode= 'rgb',
                                        subset= 'training',
                                        target_size= image_size,
                                        batch_size=batch_size )

val_gen = datagen.flow_from_dataframe(dataframe=df, 
                                        directory= path,
                                        x_col= 'file_name',
                                        y_col= 'class',
                                        class_mode= "categorical",
                                        color_mode= 'rgb',
                                        subset= 'validation',
                                        target_size= image_size,
                                        batch_size=batch_size )

test_gen = datagen.flow_from_dataframe( dataframe= test_df, 
                                        directory= test_path,
                                        x_col= 'file_name',
                                        y_col= 'class',
                                        subset= None,
                                        class_mode= "categorical",
                                        color_mode= 'rgb',
                                        target_size= image_size,
                                        batch_size= batch_size)

Found 3112 validated image filenames belonging to 95 classes.
Found 778 validated image filenames belonging to 95 classes.
Found 1039 validated image filenames belonging to 95 classes.


In [114]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

tuner.search(train_gen,
             epochs=40,
             validation_data=val_gen, 
             callbacks=[stop_early],
             verbose=1)

# Train the model
model.fit(train_gen, epochs=40, validation_data=val_gen, verbose=1)


Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
512               |224               |units
0.4               |0.4               |dropout_rate

Epoch 1/40
195/195 [==============================] - 43s 191ms/step - loss: 4.4919 - accuracy: 0.0138 - val_loss: 8.6468 - val_accuracy: 0.0000e+00
Epoch 2/40
195/195 [==============================] - 37s 191ms/step - loss: 4.3862 - accuracy: 0.0119 - val_loss: 7.5983 - val_accuracy: 0.0000e+00
Epoch 3/40
195/195 [==============================] - 41s 209ms/step - loss: 4.3712 - accuracy: 0.0132 - val_loss: 9.6739 - val_accuracy: 0.0000e+00
Epoch 4/40
195/195 [==============================] - ETA: 0s - loss: 4.3623 - accuracy: 0.0135

KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].title.set_text('Training Loss ')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train'])

axs[1].plot(history.history['accuracy'])
axs[1].title.set_text('Training Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train'])

In [ ]:
print("Evaluate on test data")
results = resnet_model.evaluate(test_gen, batch_size=16)
print("test loss, test acc:", results)

In [ ]:
#Test classes were 96 instead of 95 so i checked

In [ ]:
for i in test_classes:
    if i not in classes.keys():
        print(i)

In [ ]:
test_classes.keys()

In [ ]:
# Kendi CNN modelim
filters = 32
dropout = 0.3
batch_size = batch_size= 16
kernel_size = 3
input_shape = (100, 100, 3)
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,kernel_size=kernel_size,activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,kernel_size=kernel_size,activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters, kernel_size=kernel_size, activation='relu')(y)
# convert image to vector 
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)
outputs = Dense(num_classes, activation='softmax')(y)
 # model building by supplying inputs/outputs
cnn_model = Model(inputs=inputs, outputs= outputs)


In [ ]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(train_gen, validation_data=test_gen, epochs=20, batch_size=batch_size)
# accuracy evaluation
score = cnn_model.evaluate(train_gen, validation_data= test_gen, batch_size=batch_size, verbose=0)
print("nTest accuracy: %.1f%%" % (100.0 * score[1]))